In [15]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
import sys
import re
from argparse import Namespace

sys.path.append('../src-py/')

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [17]:
%autoreload

from utils import *
from project_debater_api import *
from mt_bart_v2 import *

In [18]:
import torch
import json

import nltk
import numpy as np
import pandas as pd

from pathlib import Path
from datasets import load_dataset, load_metric, Dataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import BartTokenizer, BartForConditionalGeneration

import matplotlib.pyplot as plt

In [19]:
from tabulate import tabulate

In [20]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [21]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [22]:
device

device(type='cuda')

In [23]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

In [37]:
def generate_ds_multi_attacks(ds, model, tokenizer, premises_clm, conclusion_clm, gen_kwargs, skip_special_tokens=True, batch_size=16, nb_counters=5):
    multi_attacks = []
    for i in range(nb_counters):
        proc_ds = ds.map(lambda x :preprocess_function(x, tokenizer, premises_clm, 'counter', conclusion_clm=conclusion_clm, conclusion_idx=i), batched=True)
        proc_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
        dataloader = torch.utils.data.DataLoader(proc_ds, batch_size=batch_size)
        attacks = generate_counters(model, tokenizer, dataloader, gen_kwargs, skip_special_tokens=skip_special_tokens)
        
        #get attacked conclusions
        attacked_conclusions = ['' if i > len(x[conclusion_clm])-1 else x[conclusion_clm][i] for x in ds]
        
        multi_attacks.append(zip(attacked_conclusions, attacks))
        
    multi_attacks = list(zip(*multi_attacks)) #convert into a list of tuples, each tuple contains nb_counters
    return multi_attacks

bertscore_metric = load_metric('bertscore')
bleu_score = load_metric('bleu')

def evaluate_gen_attacks(generated_attacks, gt_attacks, detailed=False):
    result = {
        'bleu_scores': [],
        'bert-fscores': []
    }

    bertscore_metric.add_batch(predictions=generated_attacks, references=gt_attacks)    
    bertscore_result = bertscore_metric.compute(lang='en', rescale_with_baseline=True)

    gts = [[nltk.word_tokenize(refs)] if type(refs) != list else [nltk.word_tokenize(ref) for ref in refs] for refs in gt_attacks]
    preds = [nltk.word_tokenize(x) for x in generated_attacks]

    bleu_score.add_batch(predictions=preds, references=gts)    
    bleuscore_result = bleu_score.compute()

    result['bert-fscore'] =np.mean(bertscore_result['f1'])
    result['bleu'] = bleuscore_result['bleu']
    
    if detailed:
        result['bert-fscores'] = bertscore_result['f1']
        result['bleu_scores']  = []
        
        for gt, pred in zip(gt_attacks, generated_attacks):
            #remove empty counters
            gt = [x for x in gt if x != '']
            if len(gt) == 0 or len(pred) == 0:
                print(gt, '---', pred, ' empty..')
                continue
                
            gt  = [nltk.word_tokenize(gt)] if type(gt) != list else [nltk.word_tokenize(ref) for ref in gt]
            pred= nltk.word_tokenize(pred)

            bleu_score.add_batch(predictions=[pred], references=[gt])    
            bluescore_result = bleu_score.compute()

            result['bleu_scores'].append(bluescore_result['bleu'])
        
    return result


def get_max_score(references, predictions):
    #print([references] * len(predictions))
    #print(predictions)
    if len(predictions) == 0:
        print('empty predictions..')
        return {
            'bert-fscore': 0,
            'bleu': 0
        }
    
    s = evaluate_gen_attacks( predictions, [references]*len(predictions), detailed=True)
    
    return {
        'bert-fscore': np.max(s['bert-fscores']),
        'bleu': np.max(s['bleu_scores'])
    }

----------------------

### Generate counters for multi-conclusions:

In [25]:
data_unique_path = '/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed_multi_conclusions.pkl'
data_path = '/reddit_data/conclusion_and_ca_generation/test_conclusion_all.pkl'

In [26]:
# Loading Reddit models
known_conclusion_model = BartForConditionalGeneration.from_pretrained(local_home_dir  + '/output/ca-final-models/known-conc-model/checkpoint-9500').to(device)
known_conclusion_tokenizer = BartTokenizer.from_pretrained(local_home_dir + '/output/ca-final-models/known-conc-model/checkpoint-9500')

In [27]:
unique_valid_df = pd.read_pickle(ceph_dir + data_unique_path)
all_valid_df = pd.read_pickle(ceph_dir + data_path)

In [28]:
def get_n_conclusions(post_conclusions, n=5):
    d = {x[0]: x[1] for x in post_conclusions}
    original_conc_without_prompt = d.pop('', None)
    chosen_cons = [original_conc_without_prompt] + list(d.values())[:n-1]
    return chosen_cons

In [29]:
conclusion_to_concept_dict = {item[1]: item[0] for idx, row in unique_valid_df.iterrows() for item in row['post_conclusions']}

In [34]:
unique_valid_df['post_conclusions'] = unique_valid_df.post_conclusions.apply(lambda x: get_n_conclusions(x))

In [35]:
unique_valid_df[['post', 'post_conclusions']].head()

post  \
500025                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [40]:
#Generate counters according to best parameters...
gen_kwargs = {
    "do_sample": True, 
    "max_length":100,
    "top_p":0.95, 
    "num_beams":4
}

valid_ds = Dataset.from_pandas(unique_valid_df.sample(3))
valid_ds = valid_ds.flatten_indices()

Flattening the indices:   0%|          | 0/1 [00:00<?, ?ba/s]

In [61]:
valid_ds[0]

{'post_id': 't3_aivqwb',
 'split': 'test',
 'comment_id': 't1_eequrg3',
 'title': ' MAGA kid should sue every single news outlet, blue-check celebrities, and any person that can be identified in ALL media under libel and defamation.',
 'post': ["from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life.",
  'the fact that this could have been anyone is the scariest part of all.',
  'imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran.',
  "it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything.",
  'where is the accountability?',
  "if tomorrow, someone is in the same post what we just say 'sorry fam, thought u we

In [41]:
multi_conc_attacks = generate_ds_multi_attacks(valid_ds, known_conclusion_model, known_conclusion_tokenizer, 'post', 'post_conclusions', gen_kwargs)

  0%|          | 0/1 [00:00<?, ?ba/s]

["<conclusion> the maga kid should sue every media that aired thescandal </conclusion>  <premises> from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life. the fact that this could have been anyone is the scariest part of all. imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran. it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything. where is the accountability? if tomorrow, someone is in the same post what we just say 'sorry fam, thought u were nazi but i guess not'. that's insane. i don't care about fake news, that's not the point of this cmv, the view is on the aftermath the accountability. the maga kid shoul

  0%|          | 0/1 [00:00<?, ?ba/s]

["<conclusion> Fake news should be regulated and media outlets should be fined or disciplined </conclusion>  <premises> from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life. the fact that this could have been anyone is the scariest part of all. imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran. it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything. where is the accountability? if tomorrow, someone is in the same post what we just say 'sorry fam, thought u were nazi but i guess not'. that's insane. i don't care about fake news, that's not the point of this cmv, the view is on the aftermath the accountabilit

  0%|          | 0/1 [00:00<?, ?ba/s]

["<conclusion> Major and minor media outlets should be held accountable for their negligence, especially the maga kid </conclusion>  <premises> from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life. the fact that this could have been anyone is the scariest part of all. imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran. it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything. where is the accountability? if tomorrow, someone is in the same post what we just say 'sorry fam, thought u were nazi but i guess not'. that's insane. i don't care about fake news, that's not the point of this cmv, the view is on the aft

  0%|          | 0/1 [00:00<?, ?ba/s]

["<conclusion> Online shaming of journalists is irresponsible and dangerous. It should be illegal to spread false facts, </conclusion>  <premises> from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life. the fact that this could have been anyone is the scariest part of all. imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran. it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything. where is the accountability? if tomorrow, someone is in the same post what we just say 'sorry fam, thought u were nazi but i guess not'. that's insane. i don't care about fake news, that's not the point of this cmv, the view is on the 

  0%|          | 0/1 [00:00<?, ?ba/s]

["<conclusion>  </conclusion>  <premises> from the buzzfeed debacle to the recent maga kid controversy maga kid controversy, everyone should be worried about how media falls in the trap to simple politicized and partisan nature that could destroy anyone's life. the fact that this could have been anyone is the scariest part of all. imagine being 17 years old, and for the rest of your life, being known as the maga kid that harassed a native veteran. it's despicable what the media has done but also every single holier than thou people that went with the flow to feel in any terms a little better about themselves by berating a kid without knowing anything. where is the accountability? if tomorrow, someone is in the same post what we just say 'sorry fam, thought u were nazi but i guess not'. that's insane. i don't care about fake news, that's not the point of this cmv, the view is on the aftermath the accountability. the maga kid should sue every major and minor outlet that first aired it, e

In [42]:
multi_conc_attacks

[(('the maga kid should sue every media that aired thescandal',
   'the maga kid should sue every media outlet that aired thescandal. he should be able to sue every single one of them, regardless of how much money they make.'),
  ('Fake news should be regulated and media outlets should be fined or disciplined',
   "the maga kid case is a classic example of the problem with fake news. it's a case of a media outlet sensationalizing a situation in order to make it seem like it's true. the issue is that the media has a vested interest in making it look as if it's something that happened to a real person, even if it isn't. for example, if a fake news outlet was to be banned, then it would have to be the only news outlet that"),
  ('Major and minor media outlets should be held accountable for their negligence, especially the maga kid',
   "i think the maga kid should be held accountable for his actions, but i don't think the media should be sued for their negligence. the media is not respons

In [36]:
multi_conc_attacks = [[(conclusion_to_concept_dict[c[0]], c[0], c[1]) for c in conclusions if c[0] != ''] for conclusions in multi_conc_attacks]

In [37]:
unique_valid_df['post_counters'] = multi_conc_attacks

In [39]:
unique_valid_df.to_pickle('../data/output/multi_counter_preds.pkl')

-----------

### Select best counter based on its stance towards the conclusion:

In [60]:
unique_valid_df = pd.read_pickle('../data/output/multi_counter_preds.pkl')

In [44]:
from extract_claim_targets import extract_targets

2022-05-04 16:50:28,785 loading file ../../../data-ceph/arguana/arg-generation/claim-target-tagger/model/final-model.pt
2022-05-04 16:51:07,577 SequenceTagger predicts: Dictionary with 5 tags: O, S-CT, B-CT, E-CT, I-CT


In [56]:
def get_best_counter(post_item):
    if len(post_item) == 0:
        print('found empty')
        return ''
    if len(post_item) == 1:
        print('found one item')
        return post_item[0][2]
    
    post_concepts, post_conclusions, post_counters = zip(*post_item)
    conclusions_stances = get_stances(post_concepts, post_conclusions)
    counter_stances = get_stances(post_concepts, post_counters)
    best_idx = np.argmax([abs(x[1] - x[0]) for x in zip(conclusions_stances, counter_stances)])
#     print(conclusions_stances)
#     print(counter_stances)
#     print(best_idx)
    return post_counters[best_idx]

In [58]:
#Extract targets for triplets without a target
unique_valid_df['post_counters'] = unique_valid_df.post_counters.apply(lambda post_item: [(extract_targets([x[1]])[0], x[1], x[2]) if x[0] == '' else x for x in post_item])

In [59]:
#Choose best counter
unique_valid_df['best_counter'] = unique_valid_df.post_counters.apply(lambda post_item: get_best_counter(post_item)) 

ProConClient: 100%|██████████| 3/3 [00:06<00:00,  2.01s/it]


[0.384274497628212, 0.34986964613199234, 0.003301611519418657]
[0.2939830720424652, 0.0736527144908905, 0.3280356526374817]
2


ProConClient: 100%|██████████| 5/5 [00:06<00:00,  1.20s/it]

[-0.027498960494995117, 0.005367691279388964, 0.19368769717402756, -0.7468298375606537, 0.9951292888144962]
[0.22574487328529358, -0.4603879600763321, -0.7939983643591404, -0.7098192945122719, 0.9837353897746652]
2


In [ ]:
print('done')

In [ ]:
unique_valid_df.to_pickle('../data/output/multi_counter_preds.pkl')

In [ ]:
unique_valid_df[['post_concepts', 'post_conclusions', 'post_counters', 'best_counter']].sample(10).head()

In [51]:
print('doneee')

doneee


#### Evaluate approach:

In [11]:
predictions = pd.read_pickle('../data/output/multi_counter_preds.pkl')

In [14]:
predictions[['title', 'post_conclusions', 'post_counters', 'best_counter']].head(n=20)

,title,post_conclusions,post_counters,best_counter
500025,I Don't Believe in the Big Bang,"[people who say'religions created the world' as science stand for nothing., Big Bang theory is bullshit. cmv? Atheism?, Parking violation of some standard doesn't make sense, The Seventh grade almighty god never used'so' in the scientific field of The]","[(created, people who say'religions created the world' as science stand for nothing., the big bang is a scientific theory. it's the theory of the origin of the universe. there is no scientific proof to support it. the big bang theory is the only theory that has been proven to be true so far, and it is the most widely accepted scientific theory of all time. the fact that it's a theory doesn't mean it's wrong.), (Big Bang, Big Bang theory is bullshit. cmv? Atheism?, the big bang theory is bullshit. it's not just a theory, it's the only theory that can explain the universe as we know it.), (Parking violation, Parking violation of some standard doesn't make sense, the big bang is a scientific theory. it's not a religious theory, and it's certainly not a scientific one. the fact that it's a science doesn't mean it's wrong. it just means it doesn't make sense to me.), (The Seventh, The Seventh grade almighty god never used'so' in the scientific field of The, the big bang is a scientific theory. it's the theory of how the universe came to be. it has nothing to do with religion. it is a theory of the universe being created, and it is the theory that the universe was created in 6 days. there is no evidence to support it.)]","the big bang is a scientific theory. it's the theory of how the universe came to be. it has nothing to do with religion. it is a theory of the universe being created, and it is the theory that the universe was created in 6 days. there is no evidence to support it."
534021,There is no reason for Britain to remain in the EU.,"[the ea is an island nation and britain shouldn't keep being, European Union law has no place in modern world., Island country, in general, in the world of eau, Trade bloc britain has no right to maintain its ea]","[(the ea, the ea is an island nation and britain shouldn't keep being, the eu is an island nation. britain is not an island. the eu isn't a trade bloc. it's an economic and political union. the trade bloc is not a trade union.), (European Union law, European Union law has no place in modern world., the eu is not the only thing that britain has to gain from being in the eu, but it is the most important. it is not only a trade bloc, but also a political union. britains have a lot to gain by being a part of it, and it is also the only place where they can trade freely with the rest of the world. if you want to trade with the uk, france, germany, and), (Island country, Island country, in general, in the world of eau, it's not that britain is an island nation, it's that it's a member of the eu. it's the only country in the world that doesn't have a permanent border with ireland, nor does it have a single port in ireland. it has a land border with the uk, but that's the extent of it.), (Trade bloc, Trade bloc britain has no right to maintain its ea, britain has no right to maintain its ea. britain is not an island nation. it is a member of the eu and has been since it was founded.)]","the eu is not the only thing that britain has to gain from being in the eu, but it is the most important. it is not only a trade bloc, but also a political union. britains have a lot to gain by being a part of it, and it is also the only place where they can trade freely with the rest of the world. if you want to trade with the uk, france, germany, and"
415645,The Trump Administration Would Be Significantly More Harmful to US Domestic and International Policy W/out Moderates Like Mattis,"[[MMr. mattis] is much better than trump., Human voice speech can be used to spread news., United States Secretary of Defense mattis has prevented most harm from going rogue and/or creat

In [13]:
all_valid_df = pd.read_pickle(ceph_dir + data_path)
#Collect references
arg_counters = all_valid_df.groupby('post_id').agg({
        'counter': lambda x: [' '.join(c) for c in x]
    }).reset_index()

arg_counters = pd.Series(arg_counters.counter.values, index=arg_counters.post_id).to_dict()
#remove pot comments
arg_counters = {comments[0]: [x for x in comments[1] if 'i am a bot, and this action was performed automatically.' not in x] for comments in arg_counters.items()}

In [14]:
predictions['all_counters'] = predictions['post_id'].apply(lambda x: arg_counters[x])
predictions['all_counters'] = predictions.all_counters.apply(lambda claims: [c for c in claims if c !=''])
predictions = predictions[predictions.all_counters.map(len) > 0]

In [16]:
results = evaluate_gen_attacks(predictions.best_counter.tolist(), predictions.all_counters, detailed=True)

In [17]:
results

{'bleu_scores': [0.0,
  0.33856770718244344,
  0.0,
  0.0,
  0.1830569850729261,
  0.0,
  0.16528724836151193,
  0.3398743411792008,
  0.19314131101877907,
  0.0,
  0.0,
  0.0,
  0.13347995017753458,
  0.21754170358046823,
  0.0,
  0.0,
  0.0,
  0.15396201894664285,
  0.47182058358622586,
  0.4760654911341394,
  0.0,
  0.0,
  0.28922279090187153,
  0.0,
  0.0,
  0.0,
  0.13702591700752495,
  0.0,
  0.0,
  0.0,
  0.0,
  0.2314734800742452,
  0.0,
  0.17194857771873984,
  0.0,
  0.0,
  0.0,
  0.26996383313419803,
  0.0,
  0.19200560079696705,
  0.08324315537058004,
  0.19324538521785078,
  0.0,
  0.41185624980531704,
  0.18347190231449206,
  0.0,
  0.0,
  0.12149178526903522,
  0.4047499510658001,
  0.48440807796000046,
  0.0,
  0.2851899588016996,
  0.280256932175153,
  0.3524827315803765,
  0.1118762947940517,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.09973552777256603,
  0.10802381905426098,
  0.0,
  0.0,
  0.12896149335525262,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.11603242202982009,
  